In [1]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import Ridge
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from ml_logic.data_preprocessing import clean_data, resample_pings
from ml_logic.model_evaluation import evaluate_horizon
from ml_logic.feature_engineering import create_time_series_features
from ml_logic.metric import position_extrapolation, haversine_mae

from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Hyperparameter Tuning

## Objective

Optimize hyperparameters for LightGBM and Ridge models using **GridSearchCV** with cross-validation to find the best configuration for the 480-minute prediction horizon.

**Method**: 
- GridSearchCV with GroupKFold to respect vessel groups
- Custom Haversine MAE scorer
- Compare tuned models against baseline and untuned models


In [ ]:
# Data extraction and feature creation
df = pd.read_parquet("../data/processed/ais_filtered.parquet")
df = clean_data(df)
df = resample_pings(df, interval='5min')

# 480min: optimal balance where ML significantly outperforms baseline
df_lag = create_time_series_features(df, target_horizon=480,
                                     rolling=False,
                                     advanced_features=False)


Target prediction horizon: 480 min. Number of steps: 96
Defining lag windows of 80min, 240min, 480min


In [ ]:
# Separate features, targets, and groups
X = df_lag.drop(columns=["MMSI", "BaseDateTime", "target_LAT", "target_LON"])
X_selected = X.drop(columns=['COG_lag_240min', 'COG_lag_480min', 'SOG_lag_240min'])  # From feature selection
y = df_lag[["target_LAT", "target_LON"]]
groups = df_lag["MMSI"]


In [ ]:
# Train/test split respecting vessel groups (no data leakage)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=273)
for train_idx, test_idx in gss.split(X_selected, y, groups):
    X_sel_train, X_sel_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    #need to isolate a group on the train set for future crossval
    groups_train = groups.iloc[train_idx]
    groups_test = groups.iloc[test_idx]

print(f"Train: {len(X_sel_train)} samples, {groups_train.nunique()} vessels")
print(f"Test: {len(X_sel_test)} samples, {groups_test.nunique()} vessels\n")

Train: 748405 samples, 729 vessels
Test: 180459 samples, 183 vessels



## 1. Setup

Define base model parameters and custom scorer. The scorer uses Haversine distance MAE, which is our target metric.


In [ ]:
# Base parameters
lgbm_params = {
    'n_estimators': 200,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 273,
    'n_jobs': -1,
    'verbose': -1
}

# Custom scorer: Haversine MAE (greater_is_better=False means lower is better)
haversine_scorer = make_scorer(haversine_mae, greater_is_better=False)


In [ ]:
# Wrap LightGBM for multi-output regression (LAT, LON)
lgbm = MultiOutputRegressor(LGBMRegressor(**lgbm_params))

## 2. LightGBM Hyperparameter Tuning

**Grid search strategy**: 
- Focus on regularization parameters (max_depth, min_child_samples, reg_lambda) since the problem is relatively linear
- Lower max_depth values to prevent overfitting
- Test different learning rates and n_estimators combinations




In [ ]:
# Grid search: 3×3×2×3×3 = 162 combinations × 3 folds = 486 fits
param_grid = {
    'estimator__n_estimators': [200, 300, 400],
    'estimator__max_depth': [4, 6, 8],
    'estimator__learning_rate': [0.05, 0.1],
    'estimator__min_child_samples': [10, 20, 40],
    'estimator__reg_lambda': [0, 0.1, 0.5]
}

search = GridSearchCV(
    lgbm,
    param_grid=param_grid,
    cv=GroupKFold(n_splits=3),  # Respect vessel groups
    scoring=haversine_scorer,
    n_jobs=-1,
    verbose=2
)


In [ ]:
# Fit grid search (takes ~20-25 minutes)
search.fit(X_sel_train, y_train, groups=groups_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] END estimator__learning_rate=0.05, estimator__max_depth=4, estimator__min_child_samples=10, estimator__n_estimators=200, estimator__reg_lambda=0.5; total time=  15.4s
[CV] END estimator__learning_rate=0.05, estimator__max_depth=4, estimator__min_child_samples=10, estimator__n_estimators=200, estimator__reg_lambda=0.5; total time=  16.1s
[CV] END estimator__learning_rate=0.05, estimator__max_depth=4, estimator__min_child_samples=10, estimator__n_estimators=200, estimator__reg_lambda=0.5; total time=  16.2s
[CV] END estimator__learning_rate=0.05, estimator__max_depth=4, estimator__min_child_samples=10, estimator__n_estimators=200, estimator__reg_lambda=0; total time=  16.5s
[CV] END estimator__learning_rate=0.05, estimator__max_depth=4, estimator__min_child_samples=10, estimator__n_estimators=200, estimator__reg_lambda=0.1; total time=  16.5s
[CV] END estimator__learning_rate=0.05, estimator__max_depth=4, estimator__min_

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",MultiOutputRe... verbose=-1))
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'estimator__learning_rate': [0.05, 0.1], 'estimator__max_depth': [4, 6, ...], 'estimator__min_child_samples': [10, 20, ...], 'estimator__n_estimators': [200, 300, ...], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",make_scorer(h...hod='predict')
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",GroupKFold(n_...shuffle=False)
,"verbose verbose: intCont

### LightGBM Tuning Results

The best parameters found by grid search. Note that `best_score_` is negative because sklearn negates scores when `greater_is_better=False`.


In [10]:
search.best_score_

np.float64(-19.621883156908407)

In [ ]:
# Evaluate best model and compare with baseline

# Baseline: linear extrapolation
LAT_pred_baseline, LON_pred_baseline = position_extrapolation(X_sel_test, time_horizon=480)
y_pred_baseline = np.column_stack([LAT_pred_baseline, LON_pred_baseline])
mae_baseline = haversine_mae(y_test.values, y_pred_baseline)

print(f"Baseline MAE (linear extrapolation): {mae_baseline:.3f} km")
print(f"Best GridSearch CV score: {-search.best_score_:.3f} km")  # Negate because greater_is_better=False
print(f"Improvement vs baseline: {((mae_baseline - (-search.best_score_)) / mae_baseline * 100):.1f}%")

# Test best tuned model on test set
y_pred_best = search.best_estimator_.predict(X_sel_test)
mae_best_test = haversine_mae(y_test.values, y_pred_best)

print(f"\nBest model Test MAE: {mae_best_test:.3f} km")
print(f"Improvement vs baseline: {((mae_baseline - mae_best_test) / mae_baseline * 100):.1f}%")


=== BASELINE COMPARISON ===
Baseline MAE (linear extrapolation): 26.289 km
Best GridSearch CV score: 19.622 km
Improvement vs baseline: 25.4%

Best model Test MAE: 20.676 km
Improvement vs baseline: 21.3%


### Interpretation: Tuning Impact

**Results**:
- **CV score improvement**: 25.4% better than baseline (19.6 km vs 26.3 km)
- **Test set improvement**: 21.3% better than baseline (20.7 km vs 26.3 km)
- **Gap CV vs Test**: Small overfitting (20.7 km test vs 19.6 km CV), but acceptable

**Comparison with untuned model** (from notebook 4):
- Untuned LightGBM: 21.6 km
- Tuned LightGBM: 20.7 km
- **Gain from tuning**: ~0.9 km (4% improvement)

**Conclusion**: Hyperparameter tuning provides modest but consistent improvement. The tuned model is ready for final evaluation across multiple horizons in notebook 6.


In [12]:
search.best_params_

{'estimator__learning_rate': 0.05,
 'estimator__max_depth': 4,
 'estimator__min_child_samples': 20,
 'estimator__n_estimators': 200,
 'estimator__reg_lambda': 0.1}